In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv
from pydantic import BaseModel, FilePath, constr
from typing import Optional
import math
import pandas as pd
import numpy as np

In [5]:

class Config(BaseModel):
    input: FilePath
    output: constr(strip_whitespace=True, min_length=1)
    ts_col: Optional[str] = "wx_time_utc"


def _to_numeric(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


def _parse_timestamp_col(s: pd.Series) -> pd.Series:
    """
    Robust parse:
    - If already datetime-like, return as is.
    - If integer/float encoded as YYYYMMDDHHMMSS (e.g., 20250213000000),
      convert via string formatting.
    - Else attempt generic to_datetime.
    """
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, utc=True, errors="coerce")

    def parse_one(v):
        if pd.isna(v):
            return pd.NaT
        try:
            if isinstance(v, float) and not math.isnan(v):
                v = int(v)
            if isinstance(v, (int, np.integer)):
                v = str(v)
            v = str(v).split(".")[0]  # strip .0 if present
            v = v.zfill(14)  # expected length 14
            return pd.to_datetime(v, format="%Y%m%d%H%M%S", utc=True, errors="coerce")
        except Exception:
            return pd.NaT

    parsed = s.apply(parse_one)
    if parsed.isna().all():
        parsed = pd.to_datetime(s, utc=True, errors="coerce")
    return parsed


def main(cfg: Config):
    df = pd.read_csv(cfg.input)

    if "wx_wind_dir" in df.columns:
        df = df.drop(columns=["wx_wind_dir"])

    if cfg.ts_col not in df.columns:
        raise ValueError(f"Timestamp column '{cfg.ts_col}' not found in input.")

    df["ts_utc"] = _parse_timestamp_col(df[cfg.ts_col])

    numeric_cols = [
        "leg_base_duration_sec", "leg_base_distance_m",
        "leg_incident_count", "leg_incident_delay_sum_sec",
        "leg_incident_length_sum_m", "leg_total_duration_sec",
        "wx_temp_c", "wx_rain_mm", "wx_wind_kmh",
        "leg_distance_km", "leg_total_duration_hr",
        "route_total_duration_sec", "route_total_distance_m",
        "day_of_week", "hour_of_day"
    ]
    df = _to_numeric(df, numeric_cols)

    keep_cols = set(numeric_cols + ["route", "ts_utc", "wx_location"])
    df = df.loc[df["ts_utc"].notna(), [c for c in df.columns if c in keep_cols or c == "ts_utc"]]

    gb = df.groupby(["route", "ts_utc"], as_index=False)
    agg_dict = {
        "leg_base_duration_sec": "sum",
        "leg_base_distance_m": "sum",
        "leg_incident_count": "sum",
        "leg_incident_delay_sum_sec": "sum",
        "leg_incident_length_sum_m": "sum",
        "leg_total_duration_sec": "sum",
        "leg_distance_km": "sum",
        "leg_total_duration_hr": "sum",
        "wx_temp_c": "mean",
        "wx_rain_mm": "mean",
        "wx_wind_kmh": "mean",
        "route_total_duration_sec": "first",
        "route_total_distance_m": "first",
        "day_of_week": "first",
        "hour_of_day": "first",
        "wx_location": "first",
    }

    out = gb.agg(agg_dict).reset_index(drop=True)

    out["route_total_with_delay_sec"] = (
        out["leg_base_duration_sec"].fillna(0) + out["leg_incident_delay_sum_sec"].fillna(0)
    ).round().astype("Int64")

    # Check if base leg sum equals route_total_duration_sec
    mismatch = (
        out["leg_base_duration_sec"].round().astype("Int64") !=
        out["route_total_duration_sec"].round().astype("Int64")
    ).sum()
    if mismatch > 0:
        print(f"[WARN] {mismatch} rows where base leg sum != route total duration.")

    front = [
        "route", "ts_utc", "route_total_with_delay_sec",
        "route_total_duration_sec", "route_total_distance_m",
        "day_of_week", "hour_of_day",
        "wx_temp_c", "wx_rain_mm", "wx_wind_kmh", "wx_location",
        "leg_incident_count", "leg_incident_delay_sum_sec", "leg_incident_length_sum_m",
        "leg_base_duration_sec", "leg_base_distance_m",
        "leg_total_duration_sec", "leg_distance_km", "leg_total_duration_hr"
    ]
    cols = [c for c in front if c in out.columns] + [c for c in out.columns if c not in front]
    out = out[cols]

    out.to_csv(cfg.output, index=False)
    print(f"[OK] Wrote {len(out):,} rows to {cfg.output}")
    print("[OK] Columns:", ", ".join(out.columns))

In [7]:
cfg = Config(input="dummy_dataset_hourly.csv", output="final_dataset.csv")
main(cfg)

[OK] Wrote 8,688 rows to final_dataset.csv
[OK] Columns: route, ts_utc, route_total_with_delay_sec, route_total_duration_sec, route_total_distance_m, day_of_week, hour_of_day, wx_temp_c, wx_rain_mm, wx_wind_kmh, wx_location, leg_incident_count, leg_incident_delay_sum_sec, leg_incident_length_sum_m, leg_base_duration_sec, leg_base_distance_m, leg_total_duration_sec, leg_distance_km, leg_total_duration_hr


In [9]:
df = pd.read_csv(r"C:\Users\DELL\Desktop\AI Projects\Langchain_proj\Logistic_Agent_Planner\final_dataset.csv")

In [10]:
df.columns

Index(['route', 'ts_utc', 'route_total_with_delay_sec',
       'route_total_duration_sec', 'route_total_distance_m', 'day_of_week',
       'hour_of_day', 'wx_temp_c', 'wx_rain_mm', 'wx_wind_kmh', 'wx_location',
       'leg_incident_count', 'leg_incident_delay_sum_sec',
       'leg_incident_length_sum_m', 'leg_base_duration_sec',
       'leg_base_distance_m', 'leg_total_duration_sec', 'leg_distance_km',
       'leg_total_duration_hr'],
      dtype='object')

In [11]:
df['is_weekend'] = df['day_of_week'].isin([6, 7]).astype(int)

# Time of day segment
def time_of_day(hour):
    if 5 <= hour < 10:
        return 'morning_peak'
    elif 10 <= hour < 16:
        return 'afternoon'
    elif 16 <= hour < 20:
        return 'evening_peak'
    else:
        return 'night'

df['time_of_day_segment'] = df['hour_of_day'].apply(time_of_day)

# --- WEATHER FEATURES ---
df['rain_flag'] = (df['wx_rain_mm'] > 0).astype(int)
df['wind_strength'] = pd.cut(df['wx_wind_kmh'],
                             bins=[-1, 20, 40, float('inf')],
                             labels=['low', 'medium', 'high'])
# Feels like temperature (simple adjustment for wind)
df['feels_like_temp'] = df['wx_temp_c'] - (df['wx_wind_kmh'] * 0.1)

# Extreme weather flag
df['extreme_weather_flag'] = ((df['wx_rain_mm'] > 20) | (df['wx_wind_kmh'] > 50)).astype(int)

# --- CALCULATED FEATURES ---
df['avg_leg_speed_kmh'] = (df['leg_base_distance_m'] / 1000) / (df['leg_base_duration_sec'] / 3600)
df['delay_ratio'] = df['leg_incident_delay_sum_sec'] / df['leg_total_duration_sec']

# --- INTERACTION FEATURES ---
df['weather_delay_interaction'] = df['wx_rain_mm'] * df['delay_ratio']
df['incident_weather_interaction'] = df['leg_incident_count'] * df['wx_wind_kmh']
df['time_weather_interaction'] = df['is_weekend'] * df['rain_flag']

In [12]:
pd.set_option('display.max_columns', None)
df.head()

,route,ts_utc,route_total_with_delay_sec,route_total_duration_sec,route_total_distance_m,day_of_week,hour_of_day,wx_temp_c,wx_rain_mm,wx_wind_kmh,wx_location,leg_incident_count,leg_incident_delay_sum_sec,leg_incident_length_sum_m,leg_base_duration_sec,leg_base_distance_m,leg_total_duration_sec,leg_distance_km,leg_total_duration_hr,is_weekend,time_of_day_segment,rain_flag,wind_strength,feels_like_temp,extreme_weather_flag,avg_leg_speed_kmh,delay_ratio,weather_delay_interaction,incident_weather_interaction,time_weather_interaction
0,Hume_Highway,2025-02-13 00:00:00+00:00,42347,37561,867226,3,0,22.766667,1.700000,7.633333,Melbourne (Olympic Park),131,4786,84391.194604,37561,867226,42347,867.226,11.763056,0,night,1,low,22.003333,0,83.11849,0.113019,0.192132,999.966667,0
1,Hume_Highway,2025-02-13 01:00:00+00:00,42873,37561,867226,3,1,21.433333,0.000000,13.666667,Melbourne (Olympic Park),80,5312,64681.090859,37561,867226,42873,867.226,11.909167,0,night,0,low,20.066667,0,83.11849,0.123901,0.000000,1093.333333,0
2,Hume_Highway,2025-02-13 02:00:00+00:00,39678,37561,867226,3,2,22.800000,2.433333,9.333333,Melbourne (Olympic Park),111,2117,70604.977373,37561,867226,39678,867.226,11.021667,0,night,1,low,21.866667,0,83.11849,0.053355,0.129829,1036.000000,0
3,Hume_Highway,2025-02-13 03:00:00+00:00,38860,37561,867226,3,3,22.366667,2.466667,13.700000,Melbourne (Olympic Park),126,1299,70498.121153,37561,867226,38860,867.226,10.794444,0,night,1,low,20.996667,0,83.11849,0.033428,0.082455,1726.200000,0
4,Hume_Highway,2025-02-13 04:00:00+00:00,45093,37561,867226,3,4,23.233333,2.933333,14.266667,Melbourne (Olympic Park),141,7532,102510.483745,37561,867226,45093,867.226,12.525833,0,night,1,low,21.806667,0,83.11849,0.167033,0.489962,2011.600000,0


In [19]:
df.to_csv("final_dataset_v1.csv",index = False)